In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [2]:
df = pd.read_csv("./payday_Lender_Loc.csv")

In [3]:
data = []

In [4]:
pattern = re.compile(r'-\D')
for addr in df.address:
    info = addr
    if pattern.findall(addr):
        line = addr.split('-')
        info = line[0]
    df.address[df.address == addr] = info

In [5]:
# Create a webdriver for the purpose of determining the geolocation of the addresses
driver = webdriver.Chrome()
driver.get("https://gps-coordinates.org/")

In [6]:
isRefreshTime = 0
inputBox = driver.find_element_by_xpath("//*[@id='address']")

In [7]:
for _,row in df.iterrows():
    addr = row.address
    
    # Prevent website freezing 
    if isRefreshTime == 15:
        driver.refresh()
        time.sleep(1)
        inputBox = driver.find_element_by_xpath("//*[@id='address']")
        inputBox.clear()
        isRefreshTime = 0
    
    # Find the input box and the search button
    findBtn = driver.find_element_by_xpath("//*[@id='btnGetGpsCoordinates']")
    
    # Input the address into the input textfield
    inputBox.send_keys(addr)
    findBtn.click()
    
    # Wait for the AJAX to process and finish finding the geolocation
    time.sleep(3)
    
    # Locate the longitude and latitude tags and get their values
    lat = driver.find_element_by_xpath("//*[@id='latitude']")
    lng = driver.find_element_by_xpath("//*[@id='longitude']")
    
    # Put the latitude and longitude into the list
    data.append((row.lender, addr, lat.get_attribute("value"), lng.get_attribute("value")))
    
    inputBox.clear()
    isRefreshTime += 1

In [12]:
LatLongDf = pd.DataFrame(data, ['Lender', 'Address', 'Latitude', 'Longitude'])

ValueError: Shape of passed values is (4, 590), indices imply (4, 4)

In [10]:
df.to_csv('Payday_Lender_LATLONG.csv', index=False, encoding='utf-8')

AttributeError: 'list' object has no attribute 'to_csv'